In [2]:
import os,sys
import pandas as pd
import numpy as np
from scipy import stats
# importing ploting libraries
import matplotlib.pyplot as plt
#importing seaborn for statistical plots
import seaborn as sns
sns.distributions._has_statsmodels=False
# To enable plotting graphs in Jupyter notebook
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# calculate accuracy measures and confusion matrix
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, classification_report, confusion_matrix,auc
from sklearn.metrics import recall_score,precision_score, f1_score,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# Set color code,font scale..
sns.set(color_codes=True,rc={'figure.figsize':(35.0,35.0)},font_scale=1)
%matplotlib inline 

# Let's load the packages
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt

df1 = pd.read_csv(r"frequency_domain_features_train_v1.csv")
df2 = pd.read_csv(r"heart_rate_non_linear_features_train_V1.csv")
df3 = pd.read_csv(r"time_domain_features_train_V1.csv")

object_columns_list = list(df1.select_dtypes(include='object').columns)

for object_column in object_columns_list:
    df1[object_column] = df1[object_column].astype(str)

In [3]:
mid_res = pd.merge(df1,df2,how='outer',on='uuid') 
dataframe_hrv = pd.merge(mid_res,df3,how='outer',on='uuid')

In [4]:
dataframe_hrv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369289 entries, 0 to 369288
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   uuid               369289 non-null  object 
 1   VLF_PCT            369289 non-null  float64
 2   LF_PCT             369289 non-null  float64
 3   LF_NU              369289 non-null  float64
 4   HF_PCT             369289 non-null  float64
 5   HF_NU              369289 non-null  float64
 6   TP                 369289 non-null  float64
 7   LF_HF              369289 non-null  float64
 8   sampen             369289 non-null  float64
 9   higuci             369289 non-null  float64
 10  condition          369289 non-null  object 
 11  SD1_SD2 Ratio      369289 non-null  float64
 12  SDRR_RMSSD         369289 non-null  float64
 13  HR                 369289 non-null  float64
 14  pNN50              369289 non-null  float64
 15  KURT               369289 non-null  float64
 16  SK

In [5]:
dataframe_hrv = dataframe_hrv.drop('uuid', axis=1)

In [6]:
values_replace = {
    "condition" : {"interruption" :1,"no stress":2,"time pressure":3 }
}
dataframe_hrv = dataframe_hrv.replace(values_replace)

In [7]:
dataframe_hrv.head()

,VLF_PCT,LF_PCT,LF_NU,HF_PCT,HF_NU,TP,LF_HF,sampen,higuci,condition,SD1_SD2 Ratio,SDRR_RMSSD,HR,pNN50,KURT,SKEW,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,72.203287,27.375666,98.485263,0.421047,1.514737,3686.666157,65.018055,2.139754,1.163485,2,0.055267,9.063146,69.499952,0.533333,-0.856554,0.335218,0.007969,2.143342,-0.856554,0.335218
1,76.975728,22.954139,99.695397,0.070133,0.304603,3006.487251,327.296635,2.174499,1.084711,1,0.079995,6.272369,64.363150,0.000000,-0.408190,-0.155286,0.004769,2.930855,-0.408190,-0.155286
2,51.152225,48.335104,98.950472,0.512671,1.049528,2685.879461,94.280910,2.135350,1.176315,1,0.096969,5.182201,67.450066,0.200000,0.351789,-0.656813,0.008716,2.127053,0.351789,-0.656813
3,70.180308,29.290305,98.224706,0.529387,1.775294,3434.520980,55.328701,2.178341,1.179688,2,0.087336,5.748591,68.809562,0.133333,-0.504947,-0.386138,0.008660,2.050988,-0.504947,-0.386138
4,43.918366,54.242160,96.720007,1.839473,3.279993,2621.175204,29.487873,2.221121,1.249612,2,0.154935,3.266724,74.565728,0.200000,-0.548408,-0.154252,0.013055,1.816544,-0.548408,-0.154252


In [8]:
dataframe_hrv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369289 entries, 0 to 369288
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   VLF_PCT            369289 non-null  float64
 1   LF_PCT             369289 non-null  float64
 2   LF_NU              369289 non-null  float64
 3   HF_PCT             369289 non-null  float64
 4   HF_NU              369289 non-null  float64
 5   TP                 369289 non-null  float64
 6   LF_HF              369289 non-null  float64
 7   sampen             369289 non-null  float64
 8   higuci             369289 non-null  float64
 9   condition          369289 non-null  int64  
 10  SD1_SD2 Ratio      369289 non-null  float64
 11  SDRR_RMSSD         369289 non-null  float64
 12  HR                 369289 non-null  float64
 13  pNN50              369289 non-null  float64
 14  KURT               369289 non-null  float64
 15  SKEW               369289 non-null  float64
 16  SD

In [9]:
dataframe_hrv.describe()

,VLF_PCT,LF_PCT,LF_NU,HF_PCT,HF_NU,TP,LF_HF,sampen,higuci,condition,SD1_SD2 Ratio,SDRR_RMSSD,HR,pNN50,KURT,SKEW,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
count,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000
mean,64.289242,34.095182,95.566718,1.615576,4.433282,3185.356025,115.977200,2.062471,1.182292,1.888724,0.086593,7.396597,73.941824,0.866001,0.523235,0.041628,0.009701,2.006817,0.523235,0.041628
std,16.774844,16.040290,4.123365,1.761073,4.123365,1923.227187,360.855129,0.206999,0.062192,0.667694,0.035197,5.143834,10.337453,0.990189,1.790348,0.699522,0.003897,0.375845,1.790348,0.699522
min,19.031219,2.165119,69.879083,0.002150,0.012825,377.692795,2.319952,0.434576,1.033984,1.000000,0.009142,2.660381,48.737243,0.000000,-1.894820,-2.136278,0.003220,1.169342,-1.894820,-2.136278
25%,52.909877,22.305936,93.645734,0.346803,1.228054,1828.147788,14.737458,2.032977,1.139929,1.000000,0.063277,4.541896,66.715776,0.000000,-0.352783,-0.359291,0.006984,1.749801,-0.352783,-0.359291
50%,66.350237,32.047025,96.643140,1.039513,3.356860,2796.856587,28.789747,2.134214,1.174293,2.000000,0.084329,5.952112,74.217809,0.466667,0.040736,-0.060966,0.008691,1.934416,0.040736,-0.060966
75%,76.825032,44.647115,98.771946,2.245115,6.354266,4052.260157,80.429614,2.181929,1.223621,2.000000,0.110795,7.919841,80.334937,1.466667,0.722833,0.282417,0.011460,2.221232,0.722833,0.282417
max,97.738848,77.928847,99.987175,13.095664,30.120917,13390.684100,7796.443096,2.234841,1.361219,3.000000,0.191418,54.523950,113.752308,5.466667,64.088107,6.777800,0.026955,3.724134,64.088107,6.777800


In [10]:
dataframe_hrv.isna().sum()

VLF_PCT              0
LF_PCT               0
LF_NU                0
HF_PCT               0
HF_NU                0
TP                   0
LF_HF                0
sampen               0
higuci               0
condition            0
SD1_SD2 Ratio        0
SDRR_RMSSD           0
HR                   0
pNN50                0
KURT                 0
SKEW                 0
SDSD_REL_RR          0
SDRR_RMSSD_REL_RR    0
KURT_REL_RR          0
SKEW_REL_RR          0
dtype: int64

In [ ]:
columns = list(dataframe_hrv)[:]
dataframe_hrv[columns].hist(bins=100,figsize = (15,12))

In [ ]:
sns.pairplot(dataframe_hrv)

In [ ]:
dataframe_hrv.corr()

In [ ]:
sns.heatmap(dataframe_hrv.corr())

In [11]:
from sklearn import preprocessing
normalized_df = preprocessing.normalize(dataframe_hrv)
dataframe_hrv_scaled = pd.DataFrame(normalized_df)


In [12]:
dataframe_hrv_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.019567,0.007419,0.026690,0.000114,0.000410,0.999088,0.017620,0.000580,0.000315,0.000542,0.000015,0.002456,0.018835,0.000145,-0.000232,0.000091,0.000002,0.000581,-0.000232,0.000091
1,0.025424,0.007581,0.032928,0.000023,0.000101,0.993009,0.108102,0.000718,0.000358,0.000330,0.000026,0.002072,0.021258,0.000000,-0.000135,-0.000051,0.000002,0.000968,-0.000135,-0.000051
2,0.019008,0.017961,0.036769,0.000191,0.000390,0.998050,0.035034,0.000793,0.000437,0.000372,0.000036,0.001926,0.025064,0.000074,0.000131,-0.000244,0.000003,0.000790,0.000131,-0.000244
3,0.020414,0.008520,0.028571,0.000154,0.000516,0.999015,0.016094,0.000634,0.000343,0.000582,0.000025,0.001672,0.020015,0.000039,-0.000147,-0.000112,0.000003,0.000597,-0.000147,-0.000112
4,0.016730,0.020663,0.036844,0.000701,0.001249,0.998497,0.011233,0.000846,0.000476,0.000762,0.000059,0.001244,0.028405,0.000076,-0.000209,-0.000059,0.000005,0.000692,-0.000209,-0.000059


In [13]:
dataframe_hrv_cols= dataframe_hrv.columns
dataframe_hrv_cols

Index(['VLF_PCT', 'LF_PCT', 'LF_NU', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF',
       'sampen', 'higuci', 'condition', 'SD1_SD2 Ratio', 'SDRR_RMSSD', 'HR',
       'pNN50', 'KURT', 'SKEW', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'KURT_REL_RR', 'SKEW_REL_RR'],
      dtype='object')

In [14]:
dataframe_hrv_scaled.columns = dataframe_hrv_cols
dataframe_hrv_scaled.head()

,VLF_PCT,LF_PCT,LF_NU,HF_PCT,HF_NU,TP,LF_HF,sampen,higuci,condition,SD1_SD2 Ratio,SDRR_RMSSD,HR,pNN50,KURT,SKEW,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,0.019567,0.007419,0.026690,0.000114,0.000410,0.999088,0.017620,0.000580,0.000315,0.000542,0.000015,0.002456,0.018835,0.000145,-0.000232,0.000091,0.000002,0.000581,-0.000232,0.000091
1,0.025424,0.007581,0.032928,0.000023,0.000101,0.993009,0.108102,0.000718,0.000358,0.000330,0.000026,0.002072,0.021258,0.000000,-0.000135,-0.000051,0.000002,0.000968,-0.000135,-0.000051
2,0.019008,0.017961,0.036769,0.000191,0.000390,0.998050,0.035034,0.000793,0.000437,0.000372,0.000036,0.001926,0.025064,0.000074,0.000131,-0.000244,0.000003,0.000790,0.000131,-0.000244
3,0.020414,0.008520,0.028571,0.000154,0.000516,0.999015,0.016094,0.000634,0.000343,0.000582,0.000025,0.001672,0.020015,0.000039,-0.000147,-0.000112,0.000003,0.000597,-0.000147,-0.000112
4,0.016730,0.020663,0.036844,0.000701,0.001249,0.998497,0.011233,0.000846,0.000476,0.000762,0.000059,0.001244,0.028405,0.000076,-0.000209,-0.000059,0.000005,0.000692,-0.000209,-0.000059


In [ ]:
sns.pairplot(data = dataframe_hrv_scaled)

In [15]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv_scaled.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv_scaled[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
rf.fit(X_train, y_train)

C:\Users\kavya\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=0.5, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=40, n_jobs=-1, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [16]:
rf.feature_importances_

array([2.43799582e-02, 1.91060411e-02, 2.46403953e-01, 2.80073300e-02,
       7.69407492e-03, 4.95756019e-02, 3.00886063e-03, 1.61956344e-01,
       3.78170362e-01, 3.23449354e-03, 6.08321973e-04, 8.26329896e-04,
       3.44724397e-04, 9.13617009e-04, 3.42843785e-03, 3.06319622e-04,
       6.80128768e-02, 9.16498738e-04, 3.10585378e-03])

In [17]:
import pandas as pd
important_features = pd.Series(data=rf.feature_importances_,index=X_train.columns)
important_features.sort_values(ascending=False,inplace=True)
important_features

higuci               0.378170
LF_NU                0.246404
sampen               0.161956
SDRR_RMSSD_REL_RR    0.068013
TP                   0.049576
HF_PCT               0.028007
VLF_PCT              0.024380
LF_PCT               0.019106
HF_NU                0.007694
SKEW                 0.003428
condition            0.003234
SKEW_REL_RR          0.003106
LF_HF                0.003009
KURT_REL_RR          0.000916
KURT                 0.000914
SDRR_RMSSD           0.000826
SD1_SD2 Ratio        0.000608
pNN50                0.000345
SDSD_REL_RR          0.000306
dtype: float64

In [18]:
pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

0.00012692138487013392

MEDIAN_RR                  0.480408

MEAN_RR                    0.292381

HF_PCT                     0.077685

HF_NU                      0.049392

HF_LF                      0.024874

LF_HF                      0.018726

LF_NU                      0.017613

SDSD_REL_RR                0.008657

RMSSD_REL_RR               0.003677

SDRR                       0.003232

SD2                        0.003202

sampen                     0.002760

HF                         0.002426

SDRR_RMSSD                 0.002088

SKEW_REL_RR                0.001507

SDRR_REL_RR                0.001207

SDRR_RMSSD_REL_RR          0.001061

condition_no stress        0.000912

higuci                     0.000912

SKEW                       0.000884

KURT                       0.000878

SDSD                       0.000710

VLF_PCT                    0.000665

KURT_REL_RR                0.000518

LF_PCT                     0.000502

pNN25                      0.000494

TP                         0.000492

In [ ]:
selected_x_columns = ['HR', 'MEDIAN_RR', 'MEAN_RR', 'HF_PCT', 'HF_PCT', 'LF_HF', 'LF_NU','SDSD_REL_RR','RMSSD_REL_RR']
latest = dataframe_hrv[selected_x_columns]
display(latest.head(5))

# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = latest.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = latest[['HR']]

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.5, n_jobs=-1, oob_score=True)
rf.fit(X_train, y_train)

In [ ]:
pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

In [ ]:
selected_x_columns = ['HR', 'MEDIAN_RR', 'MEAN_RR', 'HF_PCT', 'HF_PCT', 'LF_HF', 'LF_NU','SDSD_REL_RR','RMSSD_REL_RR','SDRR','SD2']
latest = dataframe_hrv[selected_x_columns]
display(latest.head(5))

# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = latest.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = latest[['HR']]

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=100,n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=120,n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=250,n_jobs=-1)
rf.fit(X_train, y_train)

pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = dataframe_hrv.drop('HR', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = dataframe_hrv[['HR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

rf = RandomForestRegressor(n_estimators=300,n_jobs=-1)
rf.fit(X_train, y_train)

pred= rf.predict(X_test)
from sklearn import metrics
metrics.mean_absolute_error(y_test,pred)

In [ ]:
df1 = pd.read_csv(r"frequency_domain_features_test.csv")
df2 = pd.read_csv(r"heart_rate_non_linear_features_test.csv")
df3 = pd.read_csv(r"time_domain_features_test.csv")

object_columns_list = list(df1.select_dtypes(include='object').columns)

for object_column in object_columns_list:
    df1[object_column] = df1[object_column].astype(str)
    
mid_res = pd.merge(df1,df2,how='outer',on='uuid') 
testData = pd.merge(mid_res,df3,how='outer',on='uuid')

testData.info()

In [ ]:

# Convert categorical variable into dummy/indicator variables. As many columns will be created as distinct values
# This is also kown as one hot coding. The column names will be America, Europe and Asia... with one hot coding
testData = pd.get_dummies(testData, columns=['condition'])
pred= rf.predict(testData)


In [ ]:
pred

In [ ]:
df1 = pd.read_csv(r"frequency_domain_features_test.csv")
df2 = pd.read_csv(r"heart_rate_non_linear_features_test.csv")
df3 = pd.read_csv(r"time_domain_features_test.csv")

object_columns_list = list(df1.select_dtypes(include='object').columns)

for object_column in object_columns_list:
    df1[object_column] = df1[object_column].astype(str)
    
mid_res = pd.merge(df1,df2,how='outer',on='uuid') 
testData = pd.merge(mid_res,df3,how='outer',on='uuid')

testData.info()

In [ ]:
ids=testData['uuid']

ids.shape

In [ ]:
pred.shape

In [ ]:
df = pd.DataFrame({'UUID':ids, 'HR':pred})
df.reset_index()

In [ ]:
print(df)

In [ ]:
# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(df)